# Exploring the NASA Exoplanet Archive planetary properties data

In [7]:
import numpy as np
import pandas as pd
import os
import glob
from astropy.io import fits
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from astropy.time import Time
import lightkurve as lk
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set_context('paper')

from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
table_1 = Table.read('../data/Exoplanet_Finder/PS_2021.04.05_08.52.09.votable',format='votable')

In [3]:
df_orig = table_1.to_pandas()

In [4]:
df_orig.tail()

,pl_name,hostname,default_flag,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,soltype,pl_controv_flag,...,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,rowupdate,pl_pubdate,releasedate
29382,ups And d,ups And,0,2,3,Radial Velocity,1999,Multiple Observatories,Published Confirmed,0,...,-0.023,2.859,0.274,-0.274,3.98687,0.008937,-0.008937,2014-08-21,2004-01,2014-08-21
29383,ups And d,ups And,1,2,3,Radial Velocity,1999,Multiple Observatories,Published Confirmed,0,...,-0.023,2.859,0.274,-0.274,3.98687,0.008937,-0.008937,2019-01-28,2011-01,2019-01-31
29384,ups And d,ups And,0,2,3,Radial Velocity,1999,Multiple Observatories,Published Confirmed,0,...,-0.023,2.859,0.274,-0.274,3.98687,0.008937,-0.008937,2014-09-18,2007-01,2014-09-18
29385,xi Aql b,xi Aql,1,1,1,Radial Velocity,2007,Okayama Astrophysical Observatory,Published Confirmed,0,...,-0.023,2.171,0.220,-0.220,4.42501,0.003837,-0.003837,2014-05-14,2008-06,2014-05-14
29386,xi Aql b,xi Aql,0,1,1,Radial Velocity,2007,Okayama Astrophysical Observatory,Published Confirmed,0,...,-0.023,2.171,0.220,-0.220,4.42501,0.003837,-0.003837,2014-07-23,2011-08,2014-07-23


In [5]:
list(df_orig.columns)

['pl_name',
 'hostname',
 'default_flag',
 'sy_snum',
 'sy_pnum',
 'discoverymethod',
 'disc_year',
 'disc_facility',
 'soltype',
 'pl_controv_flag',
 'pl_refname',
 'pl_orbper',
 'pl_orbpererr1',
 'pl_orbpererr2',
 'pl_orbperlim',
 'pl_orbsmax',
 'pl_orbsmaxerr1',
 'pl_orbsmaxerr2',
 'pl_orbsmaxlim',
 'pl_rade',
 'pl_radeerr1',
 'pl_radeerr2',
 'pl_radelim',
 'pl_radj',
 'pl_radjerr1',
 'pl_radjerr2',
 'pl_radjlim',
 'pl_bmasse',
 'pl_bmasseerr1',
 'pl_bmasseerr2',
 'pl_bmasselim',
 'pl_bmassj',
 'pl_bmassjerr1',
 'pl_bmassjerr2',
 'pl_bmassjlim',
 'pl_bmassprov',
 'pl_orbeccen',
 'pl_orbeccenerr1',
 'pl_orbeccenerr2',
 'pl_orbeccenlim',
 'pl_insol',
 'pl_insolerr1',
 'pl_insolerr2',
 'pl_insollim',
 'pl_eqt',
 'pl_eqterr1',
 'pl_eqterr2',
 'pl_eqtlim',
 'ttv_flag',
 'st_refname',
 'st_spectype',
 'st_teff',
 'st_tefferr1',
 'st_tefferr2',
 'st_tefflim',
 'st_rad',
 'st_raderr1',
 'st_raderr2',
 'st_radlim',
 'st_mass',
 'st_masserr1',
 'st_masserr2',
 'st_masslim',
 'st_met',
 'st_me

In [6]:
df=df_orig[['hostname','st_teff','st_rad','st_mass','st_logg','sy_refname','ra','dec']]
df

,hostname,st_teff,st_rad,st_mass,st_logg,sy_refname,ra,dec
0,11 Com,NaN,NaN,2.60,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,185.178772,17.793251
1,11 Com,4742.0,19.00,2.70,2.31,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,185.178772,17.793251
2,11 UMi,4340.0,24.08,1.80,1.60,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,229.274597,71.823944
3,11 UMi,NaN,NaN,1.70,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,229.274597,71.823944
4,11 UMi,4213.0,29.79,2.78,1.93,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,229.274597,71.823944
...,...,...,...,...,...,...,...,...
29382,ups And,NaN,NaN,NaN,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,24.198353,41.403816
29383,ups And,NaN,1.56,1.30,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,24.198353,41.403816
29384,ups And,NaN,NaN,NaN,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,24.198353,41.403816
29385,xi Aql,4780.0,12.00,2.20,2.66,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,298.562439,8.461105


In [8]:
hat_p_32_coord = '02 04 10.2775457769 +46 41 16.210382751'

In [14]:
coordinates = SkyCoord(hat_p_32_coord,unit=(u.hourangle,u.deg))

In [15]:
catalog = SkyCoord(ra=df.ra.values*u.degree, dec=df.dec.values*u.degree)
idx, d2d, d3d = coordinates.match_to_catalog_sky(catalog)

In [16]:
idx #returns index with number within list for desired object

array(766, dtype=int64)

In [17]:
df.iloc[766]

hostname                                               HAT-P-32
st_teff                                                  6269.0
st_rad                                                     1.22
st_mass                                                     NaN
st_logg                                                     NaN
sy_refname    <a refstr=STASSUN_ET_AL__2019 href=https://ui....
ra                                                    31.042761
dec                                                   46.687851
Name: 766, dtype: object

In [42]:
coordinates=SkyCoord.from_name('HAT-P-32b')

In [43]:
sep = coordinates.separation(catalog) #list of how far apart all objects are

In [44]:
matches = sep < 10*u.arcsec

In [45]:
matches.sum() #boolean

9

In [46]:
df[matches]

,hostname,st_teff,st_rad,st_mass,st_logg,sy_refname,ra,dec
758,HAT-P-32,NaN,NaN,1.16,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,31.042761,46.687851
759,HAT-P-32,6001.0,1.37,1.13,4.22,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,31.042761,46.687851
760,HAT-P-32,NaN,1.22,1.16,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,31.042761,46.687851
761,HAT-P-32,3187.0,NaN,NaN,4.94,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,31.042761,46.687851
762,HAT-P-32,NaN,NaN,NaN,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,31.042761,46.687851
763,HAT-P-32,6207.0,1.22,1.16,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,31.042761,46.687851
764,HAT-P-32,6207.0,1.19,1.09,4.33,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,31.042761,46.687851
765,HAT-P-32,6207.0,1.22,1.16,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,31.042761,46.687851
766,HAT-P-32,6269.0,1.22,NaN,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,31.042761,46.687851


In [20]:
idx, d2d, d3d = coordinates.match_to_catalog_sky(catalog)

In [27]:
coordinates

<SkyCoord (ICRS): (ra, dec) in deg
    (24.19934235, 41.40545674)>

In [26]:
catalog.search_around_sky([coordinates],seplimit=5*u.arcsec)

AttributeError: 'list' object has no attribute 'isscalar'